# Baseline

Let's build some simple baselines from which we can frame the results we're getting out of DM and DITTO.

Base lines:

1. Cosine Similarity >= T

In [9]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [1]:
dataset_dir = "examples/sample_data/wdc"

In [5]:
train_df = pd.read_json(os.path.join(dataset_dir, 'train.json.gz'), compression='gzip', lines=True)
print('train:', train_df.shape)
val_df = pd.read_json(os.path.join(dataset_dir, 'val.json.gz'), compression='gzip', lines=True)
print('val:', val_df.shape)
test_df = pd.read_json(os.path.join(dataset_dir, 'train.json.gz'), compression='gzip', lines=True)
print('test:', test_df.shape)

train: (33943, 22)
val: (8486, 22)
test: (33943, 22)


In [6]:
val_df.head()

,left_id,left_title,left_description,left_brand,left_price,left_specTableContent,left_keyValuePairs,left_category,left_cluster_id,left_identifiers,...,right_description,right_brand,right_price,right_specTableContent,right_keyValuePairs,right_category,right_cluster_id,right_identifiers,label,pair_id
0,6638421,hogan sneakers interactive aus wildleder in na...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,6638421#2447385
1,6987609,hogan interactive sneaker in dark blue suede,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,6987609#8305739
2,11265724,hogan interactive lav h spezzata ricamo hxw00n...,sneakers in nabuk con allacciatura frontale et...,hogan,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],...,None,None,None,None,None,Shoes,13980798,[{'/sku': '[hxw00n0s3609keu810]'}],1,11265724#6987609
3,1007757,2017 air jordan 11 midnight navy 378037 123 ne...,None,None,None,None,None,Shoes,3177224,[{'/sku': '[378037123]'}],...,None,None,None,None,None,Shoes,3177224,[{'/sku': '[378037123]'}],1,1007757#1179969
4,7157206,air jordan 11 retro blue white 378037 123 foot...,p the strong air jordan 11 retro strong is a r...,None,None,None,None,Shoes,3177224,[{'/productID': '[378037123]'}],...,None,None,None,None,None,Shoes,3177224,[{'/sku': '[378037123]'}],1,7157206#15652127


In [8]:
model = SentenceTransformer('stsb-mpnet-base-v2')

In [50]:
def cosine_sim_model(left_title, right_title, T=0.85):
    embeddings = model.encode([left_title, right_title], convert_to_tensor=True, show_progress_bar=False)
    return int(float(util.pytorch_cos_sim(embeddings[0], embeddings[1])[0]) >= T)
    return util.pytorch_cos_sim(embeddings[0], embeddings[1]) >= T

In [51]:
%%time
cosine_sim_model("mitchell took his cat for a walk", "mitch took his dog for a walk")

CPU times: user 122 ms, sys: 59.8 ms, total: 182 ms
Wall time: 193 ms


0

In [52]:
y_pred = []
y_val = []
for index, row in val_df.iterrows():

  y_pred.append(cosine_sim_model(row["left_title"], row["right_title"], 0.9))
  y_val.append(row["label"])
  if index % 1000 == 0:
    print("progress: ", str(index/len(val_df)))

(array([0, 0, 0, ..., 1, 1, 1]), Int64Index([1, 0], dtype='int64'))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))

# Slow 
it would have take 27 mins to run this code locally, so I ran on a colab instance... I got these results



In [ ]:
T >= 0.85
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      7658
           1       0.41      0.19      0.26       828
    accuracy                           0.89      8486
   macro avg       0.66      0.58      0.60      8486
weighted avg       0.87      0.89      0.88      8486

In [ ]:
T >= 0.9

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      7658
           1       0.51      0.14      0.22       828

    accuracy                           0.90      8486
   macro avg       0.71      0.56      0.58      8486
weighted avg       0.87      0.90      0.88      8486